In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.1 MB/s eta 0:00:00


In [3]:
import shutil
from pathlib import Path

# Paths on your Drive
drive_root = Path('/content/drive/MyDrive/KGP_evidence')

# Local working directory
work_dir = Path('/content/llm_eval')
work_dir.mkdir(parents=True, exist_ok=True)

# Map of source→dest
files = {
    'hotpotqa_answers_qwen.json': work_dir / 'hotpotqa_answers_qwen.json',
    'hotpotqa_evidence_qwen.json': work_dir / 'hotpotqa_evidence_qwen.json',
    '2wiki_answers_qwen.json':     work_dir / '2wiki_answers_qwen.json',
    '2wiki_evidence_qwen.json':    work_dir / '2wiki_evidence_qwen.json',
}

# Copy each file
for fname, dst in files.items():
    src = drive_root / fname
    if not src.exists():
        raise FileNotFoundError(f"{src} not found in Drive.")
    shutil.copy2(src, dst)

print("All files copied to", work_dir)

All files copied to /content/llm_eval


In [4]:
import json
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer, util

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate_em(evidence_path):
    # Load retrieved evidence records
    with open(evidence_path, 'r') as f:
        data = json.load(f)
    # Sentence‐Transformer for embeddings
    emb = SentenceTransformer("sentence-transformers/multi-qa-MiniLM-L6-cos-v1", device=DEVICE)

    total = correct = 0
    total_bridge = correct_bridge = 0
    total_comp   = correct_comp   = 0

    for rec in tqdm(data, desc=f"Evaluating {evidence_path.name}"):
        ev_texts = rec['evidence']
        ev_embs  = emb.encode(ev_texts, device=DEVICE)
        for support in rec['supports']:
            total += 1
            s_emb = emb.encode(support[1], device=DEVICE)
            sims  = util.dot_score(s_emb, ev_embs).cpu().numpy().flatten()
            hit   = (sims.max() > 0.8)

            if rec['type']=='bridge':
                total_bridge += 1
                if hit: correct_bridge += 1
            else:
                total_comp += 1
                if hit: correct_comp += 1

            if hit:
                correct += 1

    return {
        'EM_all':       correct/total,
        'EM_bridge':   correct_bridge/total_bridge,
        'EM_comparison': correct_comp/total_comp
    }


In [5]:
hotpot_res = evaluate_em(work_dir / 'hotpotqa_evidence_qwen.json')
wiki_res   = evaluate_em(work_dir / '2wiki_evidence_qwen.json')

print("\n🔹 HotpotQA EM Scores:")
print(f"  • All supports:           {hotpot_res['EM_all']:.4f}")
print(f"  • Bridge‐type supports:   {hotpot_res['EM_bridge']:.4f}")
print(f"  • Comparison‐type supports: {hotpot_res['EM_comparison']:.4f}")

print("\n🔹 2Wiki EM Scores:")
print(f"  • All supports:           {wiki_res['EM_all']:.4f}")
print(f"  • Bridge‐type supports:   {wiki_res['EM_bridge']:.4f}")
print(f"  • Comparison‐type supports: {wiki_res['EM_comparison']:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Evaluating 2wiki_evidence_qwen.json: 100%|██████████| 100/100 [00:03<00:00, 30.82it/s]


🔹 HotpotQA EM Scores:
  • All supports:           0.5837
  • Bridge‐type supports:   0.4891
  • Comparison‐type supports: 0.7037

🔹 2Wiki EM Scores:
  • All supports:           0.5120
  • Bridge‐type supports:   0.4811
  • Comparison‐type supports: 0.5347
